# Implementando o Mecanismo de Atenção em Python sem uso de frameworks

Esse projeto tem como objetivo aprender como funciona o mecanismo de atenção em um modelo Transformer. 

Attention Is All You Need: https://arxiv.org/abs/1706.03762


In [ ]:
#!pip install -q -U torch

In [1]:
# Imports
import torch
from torch import nn
import warnings
warnings.filterwarnings('ignore')

### Classe que implementa um modelo Transformer para aprendizado de sequência usando PyTorch

`nn.Module` (nn = neural network) é a classe base fundamental de todos os modelos e camadas do PyTorch. Quase tudo que constrído no PyTorch herda dela: redes neurais completas, camadas individuais, blocos de atenção, embeddings, etc.

Camadas:

`nn.Embedding` é uma camada do PyTorch que transforma cada token representado por um número inteiro em um vetor denso de dimensão fixa. Ela funciona como uma tabela de lookup onde cada índice do vocabulário corresponde a um vetor treinável, permitindo que a rede aprenda representações contínuas e significativas para palavras ou tokens. Em vez de usar one-hot vectors enormes e esparsos, o embedding fornece vetores compactos que capturam relações semânticas e sintáticas durante o treinamento, servindo como a primeira etapa essencial em modelos de NLP como Transformers, RNNs e modelos de linguagem.

`nn.MultiheadAttention`: é uma camada do PyTorch que implementa o mecanismo de atenção multi-cabeça usado nos Transformers. Ela recebe vetores de entrada e aprende a calcular relações entre eles usando várias “cabeças” de atenção paralelas, onde cada cabeça foca em um tipo diferente de dependência entre tokens. Essa camada combina consultas, chaves e valores (Q, K, V), aplica atenção escalonada e depois concatena os resultados das cabeças. É uma camada completa, treinável e central para a arquitetura Transformer.

`nn.Sequential` com Linear → ReLU → Linear: isso também é uma camada, mais especificamente um bloco feed-forward totalmente conectado, típico de cada camada do Transformer. O nn.Sequential apenas organiza várias camadas em sequência: a primeira nn.Linear projeta o vetor para uma dimensão intermediária (às vezes maior), o nn.ReLU introduz não linearidade e a segunda nn.Linear traz de volta para a dimensão original. Esse bloco é aplicado posição



In [3]:
class Transformer(nn.Module):

    def __init__(self, vocab_size, embedding_dim, n_heads, n_layers, dropout):

        # Inicializa a classe base nn.Module
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.dropout = dropout

        # Camada de embedding para converter tokens em vetores densos
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Define o mecanismo de atenção multi-cabeça
        self.attention = nn.MultiheadAttention(embedding_dim, n_heads, dropout=dropout)

        # Camada feed-forward para processamento adicional
        self.feed_forward = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )

        self.out = nn.Linear(embedding_dim, vocab_size) 

        # DEfine a camada de saída final que transformará a seuqencia de saída 
        self.out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):

        # Aplica a camada de embedding
        x = self.embedding(x)   

        # Aplica o mecanismo de atenção multi-cabeça
        x = self.attention(x)

        # Aplica a camada feed-forward
        x = self.feed_forward(x)

        # Aplica a camada de saída final
        x = self.out(x)

        return x



Parâmetros: 


`b_size`: representa o número total de tokens que o modelo consegue reconhecer e gerar. Ele define o tamanho da tabela de embeddings e o número de classes no softmax final. Um vocabulário maior permite representar mais palavras e subpalavras, mas aumenta a memória e o custo computacional. Em resumo, o vocab_size determina o universo linguístico em que o modelo opera.

`embedding_dim`: é a dimensão dos vetores usados para representar cada token do vocabulário. Cada palavra é convertida em um vetor contínuo desse tamanho, que carrega informações semânticas e sintáticas. Dimensões maiores aumentam a capacidade expressiva do modelo, mas também exigem mais memória e computação. É um dos hiperparâmetros que mais influenciam a qualidade do modelo.

`n_heads`: é o número de cabeças de atenção utilizadas no mecanismo de Multi-Head Attention. Cada cabeça aprende a focar em padrões diferentes da sequência, como relações sintáticas ou dependências distantes. Ao usar várias cabeças em paralelo, o modelo capta múltiplos tipos de interação entre tokens ao mesmo tempo. Mais cabeças aumentam a capacidade, mas exigem que embedding_dim seja divisível pelo número de cabeças.

`n_layers`: indica quantas camadas empilhadas formam o modelo Transformer. Cada camada contém atenção multi-head, normalização e redes feed-forward, expandindo a profundidade e a capacidade de abstração. Mais camadas permitem que o modelo aprenda padrões mais complexos, porém aumentam tempo de treinamento e risco de overfitting. Esse parâmetro define o “tamanho” estrutural do modelo.

`dropout`: é a taxa de probabilidade usada para desligar aleatoriamente partes da rede durante o treinamento. Isso força o modelo a não depender de unidades específicas e melhora a generalização. Valores típicos variam entre 0.1 e 0.3, equilibrando regularização e desempenho. O dropout reduz overfitting e torna o modelo mais robusto.



In [18]:
modelo = Transformer(vocab_size = 1000, 
                     embedding_dim = 32, 
                     n_heads = 4, 
                     n_layers = 2, 
                     dropout = 0.5)

In [9]:
modelo.attention    

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
)

## Construindo o modelo transformer sem uso de framework

principais camadas do modelo transformer:

1 - Camada de Embedding: transforma as palavras em vetores numéricos de tamanho fixo.

2 - Mecanismo de Atenção: permite que o modelo foque em diferentes partes de entrada.

3 - Camadas encoder e decoder: processam dados sequencialmente.

4 - Camada Linear e Sofmax: para predições finais.

### Hiperparâmetros Iniciais

In [12]:
import numpy as np

In [13]:
# Dimenão do modelo
dim_model = 64

# Cria um vetor de exemplo
x = np.random.rand(10, dim_model)  # Sequência de 10 tokens

# Tamanho do vocabulário
vocab_size = 100

### Camada Embedding

A função _embedding_ é utilizada para converter entradas sequenciais em vetores densos de tamanho fixo. Esses vetores são conhecidos como embeddings e são uma parte fundamental, em especial dos modelos de PLN, pois fornecem uma representação rica e densa de palavras ou tokens, capturando informações contextuais e semânticas que são essenciais para tarefas como tradução automática, classificação de texto, entre outras.

In [83]:
# Define a função para criar uma matriz de embedding
def embedding(input, vocab_size, dim_model):
    
    # Cria uma matriz de embedding onde cada linha representa um token do vocabulário
    # A matriz é inicializada com valores aleatórios normalmente distribuídos
    embed = np.random.randn(vocab_size, dim_model)
    
    # Para cada índice de token no input, seleciona o embedding correspondente da matriz
    # Retorna um array de embeddings correspondentes à sequência de entrada
    return np.array([embed[i] for i in input])

### Mecanismos de Atenção

O mecanismo de atenção é a parte do Transformer que decide quais palavras da frase são importantes para entender cada palavra atual.

No Transformer, Q, K e V são derivados da mesma entrada em camadas de atenção do encoder, mas de entradas diferentes no decoder (Q vem da saída da camada anterior do decoder, enquanto K e V vêm da saída do encoder). O mecanismo de atenção calcula um conjunto de pontuações (usando o produto escalar entre Q e K, daí o nome "scaled dot-product attention"), aplica uma função softmax para obter pesos de atenção e usa esses pesos para ponderar os values, criando uma saída que é uma combinação ponderada das informações relevantes de entrada.

Este processo permite que o modelo dê "atenção" às partes mais relevantes da entrada para cada parte da saída, o que é especialmente útil em tarefas como tradução, onde a relevância de diferentes palavras da entrada pode variar dependendo da parte da frase que está sendo traduzida.

### Função de Ativação Softmax

A função softmax é uma função de ativação amplamente utilizada em redes neurais, especialmente em cenários de classificação, onde é importante transformar valores brutos de saída (logits) em probabilidades que somam 1. Abaixo, está o código da função softmax com comentários em cada linha explicando seu funcionamento:

In [82]:
# Função de ativação softmax
def softmax(x):
    
    # Calcula o exponencial de cada elemento do input, ajustado pelo máximo valor no input 
    # para evitar overflow numérico
    e_x = np.exp(x - np.max(x))
    
    # Divide cada exponencial pelo somatório dos exponenciais ao longo do último eixo (axis=-1)
    # O reshape(-1, 1) garante que a divisão seja realizada corretamente em um contexto multidimensional
    return e_x / e_x.sum(axis=-1).reshape(-1, 1)

### Scale Dot Product

A função scaled_dot_product_attention() é um componente do mecanismo de atenção em modelos Transformer. Ela calcula a atenção entre conjuntos de queries (Q), keys (K) e values (V). 

Essencialmente, essa função permite que o modelo dê importância diferenciada a diferentes partes da entrada, um aspecto chave que torna os modelos Transformer particularmente eficazes para tarefas de PLN e outras tarefas sequenciais.

In [ ]:
# Define a função para calcular a atenção escalada por produto escalar
def scaled_dot_product_attention(Q, K, V):
    
    # Calcula o produto escalar entre Q e a transposta de K
    matmul_qk = np.dot(Q, K.T)
    
    # Obtém a dimensão dos vetores de chave
    depth = K.shape[-1]
    
    # Escala os logits (resultado) dividindo-os pela raiz quadrada da profundidade
    logits = matmul_qk / np.sqrt(depth)
    
    # Aplica a função softmax para obter os pesos de atenção
    attention_weights = softmax(logits)
    
    # Multiplica os pesos de atenção pelos valores V para obter a saída final
    output = np.dot(attention_weights, V)
    
    # Retorna a saída ponderada
    return output

### Saída do Modelo com Operação Linear e Softmax

A função linear_and_softmax() é uma combinação de uma camada linear seguida por uma função softmax, comumente usada em modelos de aprendizado profundo, especialmente em tarefas de classificação. 

In [73]:
# Define a função que aplica uma transformação linear seguida de softmax
def linear_and_softmax(input):
    
    # Inicializa uma matriz de pesos com valores aleatórios normalmente distribuídos
    # Esta matriz conecta cada dimensão do modelo (dim_model) a cada palavra do vocabulário (vocab_size)
    weights = np.random.randn(dim_model, vocab_size)
    
    # Realiza a operação linear (produto escalar) entre a entrada e a matriz de pesos
    # O resultado, logits, é um vetor que representa a entrada transformada em um espaço de maior dimensão
    logits = np.dot(input, weights)
    
    # Aplica a função softmax aos logits
    # Isso transforma os logits em um vetor de probabilidades, onde cada elemento soma 1
    return softmax(logits)

### Construindo o Modelo Final



In [78]:
# Função do modelo final
def transformer_model(input):
    
    # Embedding
    embedded_input = embedding(input, vocab_size, dim_model)

    # Mecanismo de Atenção 
    attention_output = scaled_dot_product_attention(embedded_input, embedded_input, embedded_input)
    
    # Camada linear e softmax
    output_probabilities = linear_and_softmax(attention_output)

    # Escolhendo os índices com maior probabilidade
    output_indices = np.argmax(output_probabilities, axis=-1)
    
    return output_indices

In [79]:
dim_model = 64
seq_length = 10
vocab_size = 100

# Gerando dados aleatórios para a entrada do modelo
input_sequence = np.random.randint(0, vocab_size, seq_length)

In [80]:
print("Sequência de entrada:", input_sequence)

Sequência de entrada: [45 34 84 90 19 73 14 49  8 47]


In [114]:
output_seq = transformer_model(input_sequence)

print("Sequência de saída (índices do token previsto pelo modelo):\n",
       output_seq)


Sequência de saída (índices do token previsto pelo modelo):
 [71 58 18 86 99 72 47 15 94 57]


Esse é uma passada, ou seja, um bloco do modelo, para que o modelo tenha boa performace, precisa que tenham várias passadas para que o modelo aprenda.